In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 73.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/2

In [26]:
import torch
import torch.nn as nn
import numpy as np
from transformers import AutoModel,  AutoProcessor
from peft import LoraConfig, get_peft_model

class WhisperAudioClassifier(nn.Module):
    def __init__(self, whisper_model_name='openai/whisper-base', num_emotions=7):
        super().__init__()
        # Load Whisper model and feature extractor
        # self.whisper = WhisperModel.from_pretrained(whisper_model_name)
        # self.feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model_name)
        self.feature_extractor = AutoProcessor.from_pretrained(whisper_model_name)
        self.whisper = AutoModel.from_pretrained(whisper_model_name)
        # Freeze original Whisper parameters
        for param in self.whisper.parameters():
            param.requires_grad = False
        
        # LoRA configuration
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=['q_proj', 'v_proj'],
            lora_dropout=0.1,
            bias='none',
            task_type='FEATURE_EXTRACTION'
        )
        
        # Apply LoRA
        self.whisper = get_peft_model(self.whisper, lora_config)
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.whisper.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_emotions)
        )
    
    def forward(self, audio):
        # Convert torch tensor to numpy if needed
        if torch.is_tensor(audio):
            audio = audio.numpy()

        print(audio.shape)
        # Preprocess audio using feature extractor
        inputs = self.feature_extractor(
            audio, 
            sampling_rate=16000, 
            return_tensors='pt'
        )
        print(inputs.input_features.shape)
        
        # Extract features from Whisper
        whisper_outputs = self.whisper.encoder(
            input_features=inputs.input_features, 
            attention_mask=torch.ones(inputs.input_features.shape[:2], device=inputs.input_features.device),
            output_hidden_states=True)
        
        # feats = whisper_outputs['hidden_states'][-1
        # print(whisper_outputs["hidden_states"].shape)
        # print(feats.shape)
        # Pool features
        pooled_features = whisper_outputs.last_hidden_state.mean(dim=1)
        
        # Classify emotions
        return self.classifier(pooled_features)

# Example usage
def main():
    # Initialize model
    model = WhisperAudioClassifier(num_emotions=7)
    
    # Create a dummy audio input (numpy array)
    dummy_audio = np.random.randn(16000).astype(np.float32)  # 1 second of audio
    
    # Forward pass
    outputs = model(dummy_audio)
    print(outputs.shape)  # Should output (batch_size, num_emotions)

In [29]:
# Initialize model
model = WhisperAudioClassifier(num_emotions=7)

# Create a dummy audio input (numpy array)
dummy_audio = np.random.randn(16000).astype(np.float32)  # 1 second of audio

# Forward pass
outputs = model(dummy_audio)
print(outputs.shape)  # Should output (batch_size, num_emotions)

(16000,)
torch.Size([1, 80, 3000])
torch.Size([1, 7])


In [30]:
model


WhisperAudioClassifier(
  (whisper): PeftModelForFeatureExtraction(
    (base_model): LoraModel(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 512)
          (layers): ModuleList(
            (0-5): 6 x WhisperEncoderLayer(
              (self_attn): WhisperSdpaAttention(
                (k_proj): Linear(in_features=512, out_features=512, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=512, out_features=512, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=512, out_features=16, bias=False)
                  )
                  (lora_B): 